In [1]:
!which python
import pandas as pd
print(pd.__version__)
import sys
print(sys.version)
import inspect
print(inspect.getfile(pd))

/gpfs/bbp.cscs.ch/ssd/apps/bsd/2024-02-01/stage_externals/install_gcc-12.3.0-skylake/python-3.11.6-bj4i6m/bin/python
2.2.1
3.12.2 | packaged by Anaconda, Inc. | (main, Feb 27 2024, 17:35:02) [GCC 11.2.0]
/gpfs/bbp.cscs.ch/data/project/proj84/csaba/git/cellden/lib/python3.12/site-packages/pandas/__init__.py


**MAKE SURE THE PICKLE FILE USED HERE IS UP TO DATE!**
(It won't load with Python 3.9 or older.. )

The pickle file is created in density3_test_total_neuron_count_pickle.ipynb

The current validation script is here: 
- https://bbpgitlab.epfl.ch/molsys/personal/gene_to_ccfv3/-/blob/main/cell_atlas/validation.py?ref_type=heads

Currently it is checking for **cell, neuron, glia, astrocyte, microglia, oligodendrocyte, gad, pv, sst, vip, generic_excitatory, generic_inhibitory** numbers

In the **cerebellum, isocortex, fiber_tracts_ids, hippocampus, hippocampal_formation, thalamus, striatum, VPL, LGd, VPM, MOB** regions

Files to create:
   - .load_nrrd(cell_density)
   - neuron_density.nrrd
   - glia_density.nrrd
   - astrocyte_density.nrrd
   - microglia_density.nrrd
   - oligodendrocyte_density.nrrd
   - gad67+_density.nrrd
   - pv+_density.nrrd
   - sst+_density.nrrd
   - vip+_density.nrrd
   - Generic_Excitatory_Neuron_MType_Generic_Excitatory_Neuron_EType.nrrd
   - Generic_Inhibitory_Neuron_MType_Generic_Inhibitory_Neuron_EType.nrrd

Reconciling region ids:
- parcellation_term_label = 1.json / AllenCCF-Annotation-2020-
- parcellation_label = AllenCCF-Annotation-2020

In [2]:
import pickle
import pandas as pd
import json
from voxcell import RegionMap
# import nrrd
import numpy as np
import os
import copy

In [3]:
path = '/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/'
total_cells = pd.read_pickle(f'{path}total_cells.pickle')

# Check if 'UVU' exists
if 'UVU' not in total_cells:
    print("Non-leaf regions of the Cerebellum were removed")
else:
    print("'UVU' key exists in combined_result_dataframes")

total_cells['ACAd2/3'].head(2)

Non-leaf regions of the Cerebellum were removed


,total_cellnumber
cluster,
0023 L5/6 IT TPE-ENT Glut_1,599.331548
0039 L6 IT CTX Glut_2,708.300920


In [4]:
region_map_path ='/gpfs/bbp.cscs.ch/data/project/proj84/atlas_pipeline_runs/2024-05-15T22:44:26+02:00/hierarchy_ccfv3_l23split_barrelsplit.json'
region_map = RegionMap.load_json(region_map_path)


In [5]:
file = '/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/metadata/parcellation_to_parcellation_term_membership_extend.csv'
parcellation_annotation = pd.read_csv(file)

print(parcellation_annotation.shape)

(3489, 19)


In [6]:
regions = list(total_cells.keys())
print(f'Regions _covered_ in the brain / total cells dict of df: {len(regions)}')
# Assuming 'my_list' is your list
if 'unassigned' in regions:
    print("Region was found, will remove: unassigned.")
    regions.remove('unassigned')
    print("For now we are removing the region unassigned as we cannot place it in the hierarchy.json system")
    
if 'brainunassigned' in regions:
    print("Region was found, will remove brain-unassigned.")
    regions.remove('brainunassigned')
    print("For now we are removing the region brain-unassigned as we cannot place it in the hierarchy.json system")

if 'brain-unassigned' in regions:
    print("Region was found, will remove brain-unassigned.")
    regions.remove('brain-unassigned')
    print("For now we are removing the region brain-unassigned as we cannot place it in the hierarchy.json system") 
    
print(f"Remaining regions: {len(regions)}")
#region_info = parcellation_annotation[parcellation_annotation['cluster_as_filename'].isin(regions)]
region_info = parcellation_annotation[parcellation_annotation['parcellation_term_acronym'].isin(regions)]

region_info.shape

Regions _covered_ in the brain / total cells dict of df: 703
Region was found, will remove: unassigned.
For now we are removing the region unassigned as we cannot place it in the hierarchy.json system
Region was found, will remove brain-unassigned.
For now we are removing the region brain-unassigned as we cannot place it in the hierarchy.json system
Remaining regions: 701


(959, 19)

In [7]:
missing_regions = parcellation_annotation[~parcellation_annotation['parcellation_term_acronym'].isin(regions)]
missin_substructures = missing_regions[missing_regions['parcellation_term_set_name'] == 'substructure']
print(f"All substructures not covered by merfish slices: \n1. {missin_substructures['parcellation_term_name'].values}")

All substructures not covered by merfish slices: 
1. ['Frontal pole, layer 1' 'Retrosplenial area, dorsal part, layer 4'
 'Lobule II' 'Lobule III' 'Lobules IV-V' 'Crus 1' 'Crus 2'
 'cuneate fascicle' 'pyramidal decussation' 'direct tectospinal pathway'
 'unassigned' 'brain, unassigned' 'Field CA1, unassigned'
 'Field CA3, unassigned' 'Bed nucleus of the anterior commissure'
 'Gracile nucleus' 'Lingula (I)' 'Declive (VI)'
 'Folium-tuber vermis (VII)' 'Pyramus (VIII)' 'Uvula (IX)' 'Nodulus (X)'
 'Simple lobule' 'Paramedian lobule' 'Copula pyramidis' 'Paraflocculus'
 'Flocculus' 'vomeronasal nerve' 'central canal, spinal cord/medulla']


In [8]:
# Initialize a list to store processed data
processed_data = []

# Loop through the names
for name in missin_substructures['parcellation_term_name'].values:
    try:
        # Find regions matching the name
        matches = list(region_map.find(name, attr='name', ignore_case=True, with_descendants=True))
        
        if matches:
            # Get the first match
            first_match = int(matches[0])
            
            # Retrieve the closest regions (with ascendants)
            upstream = region_map.get(first_match, 'name', with_ascendants=True)
            closest = upstream[1]
            
            # Add processed data to the list
            processed_data.append({'id': int(first_match), 'name': name, 'closest_region':closest, 'parent_region': upstream})
        else:
            # If no matches are found, append a placeholder
            processed_data.append({'id': None, 'name': name, 'parent_region': None})
    except Exception as e:
        # Handle exceptions and add a placeholder entry
        print(f"Error processing name '{name}': {e}")
        processed_data.append({'id': None, 'name': name, 'parent_region': None})

# Create a DataFrame from the processed data
df = pd.DataFrame(processed_data)
df.to_csv('/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/missing_regions.csv')

In [9]:
#If this does not trigger every region acronym is a substructure, if it triggers, the region is either not a substructure or have higher than substructure term set name
#If it is triggered it can be because of parcellation_term_acronym / cluster_as_filename column difference
for region in regions:
    #acronym = region_info[region_info['cluster_as_filename'] == region]
    acronym = region_info[region_info['parcellation_term_acronym'] == region]
    if acronym.shape[0] == 0:
        print(f"{region} is not only a substructure! ")

region_info.shape

(959, 19)

In [10]:
# Identify duplicates in parcellation_term_acronym
duplicates = region_info[region_info.duplicated('parcellation_term_acronym', keep=False)]

# Filter to keep only rows marked as 'substructure'
substructure_filter = duplicates['parcellation_term_set_name'] == 'substructure'

# Keep substructure duplicates and non-duplicated rows
filtered_region_info = pd.concat([region_info[~region_info['parcellation_term_acronym'].isin(duplicates['parcellation_term_acronym'])], duplicates[substructure_filter]])

# Reset the index (optional, for cleaner output)
filtered_region_info = filtered_region_info.reset_index(drop=True)

filtered_region_info.shape
#This shouls be around 709

(708, 19)

## Region parcellation based on validation code

We can check which region is covered by the current validation code __registered_regions__ and which are not __missing_reg__. 
Strings to check: regions where literature data is available
Second list: remaining areas in the brain. 


In [11]:
# List of strings to check
strings_to_check = ['Cerebellum', 'Isocortex', 'fiber tracts', 'ventricular systems', 'Hippocampus', 
                    'Hippocampal formation', 'Thalamus', 'Striatum', 'VPL', 'LGd', 'VPM', 'MOB']

second_list = ['Brain stem', 'Olfactory areas', 'Cortical subplate', 'Cerebral nuclei', ]
missing_reg = []
registered_regions = []

for region in regions:
    #print(region)
    # acronym = filtered_region_info[filtered_region_info['cluster_as_filename'] == region][['parcellation_term_acronym', 'parcellation_term_name', 'parcellation_label']]
    acronym = filtered_region_info[filtered_region_info['parcellation_term_acronym'] == region][['parcellation_term_acronym', 'parcellation_term_name', 'parcellation_label']]
    long_id = acronym['parcellation_label']
    substring = "AllenCCF-Annotation-2020-"
    index = long_id.iloc[0].find(substring)
    id = long_id.iloc[0][index + len(substring):]
    # print(long_id.iloc[0], id)
    ascendants = region_map.get(int(id), "name", with_ascendants=True)
    # Assuming ascendant is your list
    found_strings = [s for s in strings_to_check if any(s in a for a in ascendants)]
    found_strings_2 = [s for s in second_list if any(s in a for a in ascendants)]
    if found_strings:
        #print(f"Found strings in the ascendant list: {found_strings}")
        registered_regions.append(id)
    elif found_strings_2:
        missing_reg.append(id)
        #print(f"Found strings in second list: {found_strings_2}")
    else:
        print(id)
        print("One region can not be placed in the brain.")
        break

## Calculate total cells and density in a region: Isocortex

Deal with regions missing from the total cells

In [12]:
isocortex = (
    region_map.find("Isocortex", attr="name", with_descendants=True)
    | region_map.find("Entorhinal area", attr="name", with_descendants=True)
    | region_map.find("Piriform area", attr="name", with_descendants=True)
)
cerebellum = region_map.find(
        "Cerebellum", attr="name", with_descendants=True
    ) | region_map.find("arbor vitae", attr="name", with_descendants=True)

fiber_tracts_ids = (
    region_map.find("fiber tracts", attr="name", with_descendants=True)
    | region_map.find("grooves", attr="name", with_descendants=True)
    | region_map.find("ventricular systems", attr="name", with_descendants=True)
)
hippocampus = (
    region_map.find("Hippocampal region", attr="name", with_descendants=True)
)
hippocampal_formation = (
    region_map.find("Hippocampal formation", attr="name", with_descendants=True)
)
thalamus = (
    region_map.find("Thalamus", attr="name", with_descendants=True)
)
striatum = (
    region_map.find("Striatum", attr="name", with_descendants=True)
)
VPL = (
    region_map.find("Ventral posterolateral nucleus of the thalamus", attr="name", with_descendants=True)
)
LGd = (
    region_map.find("Dorsal part of the lateral geniculate complex", attr="name", with_descendants=True)
)
VPM = (
    region_map.find("Ventral posteromedial nucleus of the thalamus", attr="name", with_descendants=True)
)
MOB = (
    region_map.find("Main olfactory bulb", attr="name", with_descendants=True)
)

print("Done!")

Done!


In [13]:
#This cell will complement the two cells above
Hindbrain = (
    region_map.find("Hindbrain", attr="name", with_descendants=True)
)
Midbrain = (
    region_map.find("Midbrain", attr="name", with_descendants=True)
)
Interbrain = (
    region_map.find("Interbrain", attr="name", with_descendants=True)
)
Olfactoryareas = (
    region_map.find("Olfactory areas", attr="name", with_descendants=True)
)
Corticalsubplate = (
    region_map.find("Cortical subplate", attr="name", with_descendants=True)
)
Cerebralnuclei = (
    region_map.find("Cerebral nuclei", attr="name", with_descendants=True)
)
cerebral_cortex = (
    region_map.find("Cerebral cortex", attr="name", with_descendants=True)
)
wholebrain = (
    region_map.find("root", attr="name", with_descendants=True)
)

#This is the final dictionary of all region names and their region ids
areas = {
    'wholebrain': wholebrain, 'Cerebral cortex': cerebral_cortex,
    'isocortex': isocortex, 'cerebellum': cerebellum, 'fiber_tracts_ids': fiber_tracts_ids, 
    'hippocampus': hippocampus, 'hippocampal_formation':hippocampal_formation, 
    'thalamus': thalamus, 'striatum': striatum, 'VPL': VPL, 'LGd': LGd, 'VPM': VPM, 'MOB': MOB,
    'Olfactory areas': Olfactoryareas, 'Cortical subplate': Corticalsubplate, 
    'Cerebral nuclei': Cerebralnuclei, 'Interbrain': Interbrain , 'Midbrain': Midbrain, 
    'Interbrain': Interbrain
}

In [14]:
total_cells_copy = copy.deepcopy(total_cells) 
#Here there is no filter applied from above, so it is total cells

for key, value in areas.items():
    
    keys_to_drop = ['unassigned', 'brain-unassigned']
    for region in regions:
        
        #filtered_data = region_info[(region_info['cluster_as_filename'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        filtered_data = region_info[(region_info['parcellation_term_acronym'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        id_ = filtered_data['label_numbers'].iloc[0]
        
        if int(id_) in value:
            #print(region, id_, "is in the set")
            None
        else:
            #print(id_, "is not in the set")
            keys_to_drop.append(region)

        #break    

    # Create a new dictionary without the specified keys (all leaf regions and their cell types in 1 region)
    total_cells_filtered = {key: value for key, value in total_cells_copy.items() if key not in keys_to_drop}

    # Concatenate all DataFrames into a single DataFrame
    concatenated_df = pd.concat(total_cells_filtered.values())

    # Group by all columns except the 'total_cells' column and sum the values
    summed_df = concatenated_df.groupby(concatenated_df.index).sum()
    print(f"The brain area {key} has {summed_df.shape[0]} different cell types")
    #print("Total sum of cells:", '{:,.0f}'.format(summed_df['density_mm3'].sum()))
    print("Total sum of cells:", '{:,.0f}'.format(summed_df['total_cellnumber'].sum()))

The brain area wholebrain has 5272 different cell types
Total sum of cells: 92,231,205
The brain area Cerebral cortex has 3095 different cell types
Total sum of cells: 43,858,738
The brain area isocortex has 946 different cell types
Total sum of cells: 28,535,494
The brain area cerebellum has 861 different cell types
Total sum of cells: 6,224,671
The brain area fiber_tracts_ids has 3174 different cell types
Total sum of cells: 10,371,674
The brain area hippocampus has 590 different cell types
Total sum of cells: 4,086,949
The brain area hippocampal_formation has 1743 different cell types
Total sum of cells: 6,949,049
The brain area thalamus has 1682 different cell types
Total sum of cells: 3,260,859
The brain area striatum has 1379 different cell types
Total sum of cells: 9,844,474
The brain area VPL has 81 different cell types
Total sum of cells: 117,011
The brain area LGd has 63 different cell types
Total sum of cells: 107,728
The brain area VPM has 58 different cell types
Total sum 

In [15]:
meta_path = "/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/metadata/WMB-10X/20231215/views/cell_metadata_with_cluster_annotation.csv"
metadata = pd.read_csv(meta_path, dtype={'cell_label':str})
metadata = metadata[['class', 'subclass', 'cluster']]

n_classes = ['01 IT-ET Glut', '02 NP-CT-L6b Glut', '03 OB-CR Glut',
       '04 DG-IMN Glut', '05 OB-IMN GABA', '06 CTX-CGE GABA',
       '07 CTX-MGE GABA', '08 CNU-MGE GABA', '09 CNU-LGE GABA',
       '10 LSX GABA', '11 CNU-HYa GABA', '12 HY GABA', '13 CNU-HYa Glut',
       '14 HY Glut', '15 HY Gnrh1 Glut', '16 HY MM Glut', '17 MH-LH Glut',
       '18 TH Glut', '19 MB Glut', '20 MB GABA', '21 MB Dopa',
       '22 MB-HB Sero', '23 P Glut', '24 MY Glut', '25 Pineal Glut',
       '26 P GABA', '27 MY GABA', '28 CB GABA', '29 CB Glut',]

nn_classes = ['30 Astro-Epen', '31 OPC-Oligo', '32 OEC', '33 Vascular',
       '34 Immune']

exc = ['01 IT-ET Glut', '02 NP-CT-L6b Glut', '03 OB-CR Glut',
      '04 DG-IMN Glut', '13 CNU-HYa Glut', '14 HY Glut', '15 HY Gnrh1 Glut', '16 HY MM Glut', '17 MH-LH Glut',
      '18 TH Glut', '19 MB Glut', '23 P Glut', '24 MY Glut', '25 Pineal Glut', '29 CB Glut',]
inh = ['05 OB-IMN GABA', '06 CTX-CGE GABA', '07 CTX-MGE GABA', '08 CNU-MGE GABA', '09 CNU-LGE GABA',
      '10 LSX GABA', '11 CNU-HYa GABA', '12 HY GABA', '20 MB GABA', '26 P GABA', '27 MY GABA', '28 CB GABA', ]
other = ['21 MB Dopa', '22 MB-HB Sero', ]
exci_inhib_su = exc + inh

astrotypes = ['5206 Bergmann NN_1', '5207 Astro-CB NN_1', '5208 Astro-NT NN_1',
       '5209 Astro-NT NN_1', '5210 Astro-NT NN_1', '5211 Astro-NT NN_1',
       '5212 Astro-NT NN_1', '5213 Astro-NT NN_1', '5214 Astro-NT NN_2',
       '5215 Astro-NT NN_2', '5216 Astro-NT NN_2', '5217 Astro-NT NN_2',
       '5218 Astro-TE NN_1', '5219 Astro-TE NN_1', '5220 Astro-TE NN_1',
       '5221 Astro-TE NN_1', '5222 Astro-TE NN_2', '5223 Astro-TE NN_2',
       '5224 Astro-TE NN_3', '5225 Astro-TE NN_3', '5226 Astro-TE NN_3',
       '5227 Astro-TE NN_3', '5228 Astro-TE NN_4', '5229 Astro-TE NN_5',
       '5230 Astro-TE NN_5', '5231 Astro-OLF NN_1', '5232 Astro-OLF NN_1',
       '5233 Astro-OLF NN_2', '5234 Astro-OLF NN_2',
       '5235 Astro-OLF NN_3', '5236 Astro-OLF NN_3',]

microglia = ['5312 Microglia NN_1']

oligos = ['5266 OPC NN_1', '5267 OPC NN_1',
       '5268 OPC NN_1', '5269 OPC NN_1', '5270 OPC NN_1', '5271 OPC NN_2',
       '5272 COP NN_1', '5273 COP NN_1', '5274 COP NN_1', '5275 COP NN_1',
       '5276 COP NN_1', '5277 COP NN_1', '5278 NFOL NN_2',
       '5279 NFOL NN_2', '5280 NFOL NN_2', '5281 NFOL NN_2',
       '5282 MFOL NN_3', '5283 MFOL NN_3', '5284 MOL NN_4',
       '5285 MOL NN_4', '5286 MOL NN_4', '5287 MOL NN_4', '5288 MOL NN_4',]

glia = astrotypes + microglia + oligos

neurontypes = np.unique(metadata[metadata['class'].isin(n_classes)]['cluster'].values)
nonneurontypes = np.unique(metadata[metadata['class'].isin(nn_classes)]['cluster'].values)
exctypes = np.unique(metadata[metadata['class'].isin(exc)]['cluster'].values)
inhtypes = np.unique(metadata[metadata['class'].isin(inh)]['cluster'].values)
othertypes = np.unique(metadata[metadata['class'].isin(other)]['cluster'].values)
exci_inhib_sum = np.unique(metadata[metadata['class'].isin(exci_inhib_su)]['cluster'].values)

/nvme/veraszto/2720273/ipykernel_98699/1153689541.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(meta_path, dtype={'cell_label':str})


In [16]:
total_cells_copy = copy.deepcopy(total_cells) 

for key, df in total_cells_copy.items():
    total_cells_copy[key] = df[df.index.isin(neurontypes)]

for key, value in areas.items():
    
    keys_to_drop = ['unassigned', 'brain-unassigned']
    for region in regions:
        
        #filtered_data = region_info[(region_info['cluster_as_filename'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        filtered_data = region_info[(region_info['parcellation_term_acronym'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        id_ = filtered_data['label_numbers'].iloc[0]
        
        if int(id_) in value:
            #print(region, id_, "is in the set")
            None
        else:
            #print(id_, "is not in the set")
            keys_to_drop.append(region)

        #break    

    # Create a new dictionary without the specified keys
    total_cells_filtered = {key: value for key, value in total_cells_copy.items() if key not in keys_to_drop}

    # Concatenate all DataFrames into a single DataFrame
    concatenated_df = pd.concat(total_cells_filtered.values())

    # Group by all columns except the 'total_cells' column and sum the values
    summed_df = concatenated_df.groupby(concatenated_df.index).sum()
    print(f"The brain area {key} has {summed_df.shape[0]} different neuron types")
    #print("Total sum of these cells:", '{:,.0f}'.format(summed_df['density_mm3'].sum()))
    print("Total sum of these cells:", '{:,.0f}'.format(summed_df['total_cellnumber'].sum()))

The brain area wholebrain has 5155 different neuron types
Total sum of these cells: 52,040,591
The brain area Cerebral cortex has 2989 different neuron types
Total sum of these cells: 29,125,071
The brain area isocortex has 870 different neuron types
Total sum of these cells: 19,101,627
The brain area cerebellum has 781 different neuron types
Total sum of these cells: 3,543,172
The brain area fiber_tracts_ids has 3060 different neuron types
Total sum of these cells: 2,490,673
The brain area hippocampus has 508 different neuron types
Total sum of these cells: 2,324,177
The brain area hippocampal_formation has 1653 different neuron types
Total sum of these cells: 3,890,269
The brain area thalamus has 1594 different neuron types
Total sum of these cells: 1,829,761
The brain area striatum has 1297 different neuron types
Total sum of these cells: 7,114,754
The brain area VPL has 41 different neuron types
Total sum of these cells: 51,931
The brain area LGd has 27 different neuron types
Total

In [17]:
total_cells_copy = copy.deepcopy(total_cells) 

for key, df in total_cells_copy.items():
    total_cells_copy[key] = df[df.index.isin(nonneurontypes)]

for key, value in areas.items():
    
    keys_to_drop = ['unassigned', 'brain-unassigned']
    for region in regions:
        
        #filtered_data = region_info[(region_info['cluster_as_filename'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        filtered_data = region_info[(region_info['parcellation_term_acronym'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        id_ = filtered_data['label_numbers'].iloc[0]
        
        if int(id_) in value:
            #print(region, id_, "is in the set")
            None
        else:
            #print(id_, "is not in the set")
            keys_to_drop.append(region)

        #break    

    # Create a new dictionary without the specified keys
    total_cells_filtered = {key: value for key, value in total_cells_copy.items() if key not in keys_to_drop}

    # Concatenate all DataFrames into a single DataFrame
    concatenated_df = pd.concat(total_cells_filtered.values())

    # Group by all columns except the 'total_cells' column and sum the values
    summed_df = concatenated_df.groupby(concatenated_df.index).sum()
    print(f"The brain area {key} has {summed_df.shape[0]}/{len(nonneurontypes)} different nonneuron types")
    #print("Total sum of these cells:", '{:,.0f}'.format(summed_df['density_mm3'].sum()))
    print("Total sum of these cells:", '{:,.0f}'.format(summed_df['total_cellnumber'].sum()))

The brain area wholebrain has 117/117 different nonneuron types
Total sum of these cells: 40,190,614
The brain area Cerebral cortex has 106/117 different nonneuron types
Total sum of these cells: 14,733,667
The brain area isocortex has 76/117 different nonneuron types
Total sum of these cells: 9,433,867
The brain area cerebellum has 80/117 different nonneuron types
Total sum of these cells: 2,681,499
The brain area fiber_tracts_ids has 114/117 different nonneuron types
Total sum of these cells: 7,881,000
The brain area hippocampus has 82/117 different nonneuron types
Total sum of these cells: 1,762,772
The brain area hippocampal_formation has 90/117 different nonneuron types
Total sum of these cells: 3,058,780
The brain area thalamus has 88/117 different nonneuron types
Total sum of these cells: 1,431,098
The brain area striatum has 82/117 different nonneuron types
Total sum of these cells: 2,729,721
The brain area VPL has 40/117 different nonneuron types
Total sum of these cells: 65,0

In [18]:
total_cells_copy = copy.deepcopy(total_cells) 

for key, df in total_cells_copy.items():
    total_cells_copy[key] = df[df.index.isin(exctypes)]

for key, value in areas.items():
    
    keys_to_drop = ['unassigned', 'brain-unassigned']
    for region in regions:
        
        #filtered_data = region_info[(region_info['cluster_as_filename'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        filtered_data = region_info[(region_info['parcellation_term_acronym'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        id_ = filtered_data['label_numbers'].iloc[0]
        
        if int(id_) in value:
            #print(region, id_, "is in the set")
            None
        else:
            #print(id_, "is not in the set")
            keys_to_drop.append(region)

        #break    

    # Create a new dictionary without the specified keys
    total_cells_filtered = {key: value for key, value in total_cells_copy.items() if key not in keys_to_drop}

    # Concatenate all DataFrames into a single DataFrame
    concatenated_df = pd.concat(total_cells_filtered.values())

    # Group by all columns except the 'total_cells' column and sum the values
    summed_df = concatenated_df.groupby(concatenated_df.index).sum()
    print(f"The brain area {key} has {summed_df.shape[0]} different EXC neuron types")
    #print("Total sum of these cells:", '{:,.0f}'.format(summed_df['density_mm3'].sum()))
    print("Total sum of these cells:", '{:,.0f}'.format(summed_df['total_cellnumber'].sum()))

The brain area wholebrain has 2581 different EXC neuron types
Total sum of these cells: 30,510,432
The brain area Cerebral cortex has 1608 different EXC neuron types
Total sum of these cells: 21,587,414
The brain area isocortex has 492 different EXC neuron types
Total sum of these cells: 16,526,696
The brain area cerebellum has 395 different EXC neuron types
Total sum of these cells: 2,145,798
The brain area fiber_tracts_ids has 1425 different EXC neuron types
Total sum of these cells: 1,176,058
The brain area hippocampus has 253 different EXC neuron types
Total sum of these cells: 2,063,011
The brain area hippocampal_formation has 894 different EXC neuron types
Total sum of these cells: 3,342,058
The brain area thalamus has 801 different EXC neuron types
Total sum of these cells: 1,469,170
The brain area striatum has 367 different EXC neuron types
Total sum of these cells: 298,602
The brain area VPL has 11 different EXC neuron types
Total sum of these cells: 49,316
The brain area LGd 

In [19]:
total_cells_copy = copy.deepcopy(total_cells) 

for key, df in total_cells_copy.items():
    total_cells_copy[key] = df[df.index.isin(inhtypes)]

for key, value in areas.items():
    
    keys_to_drop = ['unassigned', 'brain-unassigned']
    for region in regions:
        
        #filtered_data = region_info[(region_info['cluster_as_filename'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        filtered_data = region_info[(region_info['parcellation_term_acronym'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        id_ = filtered_data['label_numbers'].iloc[0]
        
        if int(id_) in value:
            #print(region, id_, "is in the set")
            None
        else:
            #print(id_, "is not in the set")
            keys_to_drop.append(region)

        #break    

    # Create a new dictionary without the specified keys
    total_cells_filtered = {key: value for key, value in total_cells_copy.items() if key not in keys_to_drop}

    # Concatenate all DataFrames into a single DataFrame
    concatenated_df = pd.concat(total_cells_filtered.values())

    # Group by all columns except the 'total_cells' column and sum the values
    summed_df = concatenated_df.groupby(concatenated_df.index).sum()
    print(f"The brain area {key} has {summed_df.shape[0]} different INH neuron types")
    #print("Total sum of these cells:", '{:,.0f}'.format(summed_df['density_mm3'].sum()))
    print("Total sum of these cells:", '{:,.0f}'.format(summed_df['total_cellnumber'].sum()))

The brain area wholebrain has 2499 different INH neuron types
Total sum of these cells: 21,359,205
The brain area Cerebral cortex has 1336 different INH neuron types
Total sum of these cells: 7,532,104
The brain area isocortex has 378 different INH neuron types
Total sum of these cells: 2,574,930
The brain area cerebellum has 375 different INH neuron types
Total sum of these cells: 1,397,374
The brain area fiber_tracts_ids has 1573 different INH neuron types
Total sum of these cells: 1,296,096
The brain area hippocampus has 255 different INH neuron types
Total sum of these cells: 261,166
The brain area hippocampal_formation has 739 different INH neuron types
Total sum of these cells: 545,591
The brain area thalamus has 787 different INH neuron types
Total sum of these cells: 360,448
The brain area striatum has 930 different INH neuron types
Total sum of these cells: 6,816,152
The brain area VPL has 30 different INH neuron types
Total sum of these cells: 2,615
The brain area LGd has 12 

In [20]:
total_cells_copy = copy.deepcopy(total_cells) 

for key, df in total_cells_copy.items():
    total_cells_copy[key] = df[df.index.isin(othertypes)]

for key, value in areas.items():
    
    keys_to_drop = ['unassigned', 'brain-unassigned']
    for region in regions:
        
        #filtered_data = region_info[(region_info['cluster_as_filename'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        filtered_data = region_info[(region_info['parcellation_term_acronym'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        id_ = filtered_data['label_numbers'].iloc[0]
        
        if int(id_) in value:
            #print(region, id_, "is in the set")
            None
        else:
            #print(id_, "is not in the set")
            keys_to_drop.append(region)

        #break    

    # Create a new dictionary without the specified keys
    total_cells_filtered = {key: value for key, value in total_cells_copy.items() if key not in keys_to_drop}

    # Concatenate all DataFrames into a single DataFrame
    concatenated_df = pd.concat(total_cells_filtered.values())

    # Group by all columns except the 'total_cells' column and sum the values
    summed_df = concatenated_df.groupby(concatenated_df.index).sum()
    print(f"The brain area {key} has {summed_df.shape[0]} different OTHER neuron types")
    #print("Total sum of these cells:", '{:,.0f}'.format(summed_df['density_mm3'].sum()))
    print("Total sum of these cells:", '{:,.0f}'.format(summed_df['total_cellnumber'].sum()))

The brain area wholebrain has 75 different OTHER neuron types
Total sum of these cells: 170,954
The brain area Cerebral cortex has 45 different OTHER neuron types
Total sum of these cells: 5,553
The brain area isocortex has 0 different OTHER neuron types
Total sum of these cells: 0
The brain area cerebellum has 11 different OTHER neuron types
Total sum of these cells: 0
The brain area fiber_tracts_ids has 62 different OTHER neuron types
Total sum of these cells: 18,519
The brain area hippocampus has 0 different OTHER neuron types
Total sum of these cells: 0
The brain area hippocampal_formation has 20 different OTHER neuron types
Total sum of these cells: 2,620
The brain area thalamus has 6 different OTHER neuron types
Total sum of these cells: 143
The brain area striatum has 0 different OTHER neuron types
Total sum of these cells: 0
The brain area VPL has 0 different OTHER neuron types
Total sum of these cells: 0
The brain area LGd has 0 different OTHER neuron types
Total sum of these 

In [21]:
#glia
total_cells_copy = copy.deepcopy(total_cells) 

for key, df in total_cells_copy.items():
    total_cells_copy[key] = df[df.index.isin(glia)]

for key, value in areas.items():
    
    keys_to_drop = ['unassigned', 'brain-unassigned']
    for region in regions:
        
        #filtered_data = region_info[(region_info['cluster_as_filename'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        filtered_data = region_info[(region_info['parcellation_term_acronym'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        id_ = filtered_data['label_numbers'].iloc[0]
        
        if int(id_) in value:
            #print(region, id_, "is in the set")
            None
        else:
            #print(id_, "is not in the set")
            keys_to_drop.append(region)

        #break    

    # Create a new dictionary without the specified keys
    total_cells_filtered = {key: value for key, value in total_cells_copy.items() if key not in keys_to_drop}

    # Concatenate all DataFrames into a single DataFrame
    concatenated_df = pd.concat(total_cells_filtered.values())

    # Group by all columns except the 'total_cells' column and sum the values
    summed_df = concatenated_df.groupby(concatenated_df.index).sum()
    print(f"The brain area {key} has {summed_df.shape[0]} different GLIA types")
    #print("Total sum of these cells:", '{:,.0f}'.format(summed_df['density_mm3'].sum()))
    print("Total sum of these cells:", '{:,.0f}'.format(summed_df['total_cellnumber'].sum()))

The brain area wholebrain has 55 different GLIA types
Total sum of these cells: 27,777,010
The brain area Cerebral cortex has 53 different GLIA types
Total sum of these cells: 9,524,251
The brain area isocortex has 43 different GLIA types
Total sum of these cells: 5,915,900
The brain area cerebellum has 37 different GLIA types
Total sum of these cells: 1,586,591
The brain area fiber_tracts_ids has 53 different GLIA types
Total sum of these cells: 5,576,436
The brain area hippocampus has 43 different GLIA types
Total sum of these cells: 1,306,825
The brain area hippocampal_formation has 47 different GLIA types
Total sum of these cells: 2,207,870
The brain area thalamus has 47 different GLIA types
Total sum of these cells: 1,040,690
The brain area striatum has 45 different GLIA types
Total sum of these cells: 1,887,694
The brain area VPL has 29 different GLIA types
Total sum of these cells: 51,141
The brain area LGd has 25 different GLIA types
Total sum of these cells: 27,700
The brain a

In [22]:
#astrotypes
total_cells_copy = copy.deepcopy(total_cells) 

for key, df in total_cells_copy.items():
    total_cells_copy[key] = df[df.index.isin(astrotypes)]

for key, value in areas.items():
    
    keys_to_drop = ['unassigned', 'brain-unassigned']
    for region in regions:
        
        #filtered_data = region_info[(region_info['cluster_as_filename'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        filtered_data = region_info[(region_info['parcellation_term_acronym'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        id_ = filtered_data['label_numbers'].iloc[0]
        
        if int(id_) in value:
            #print(region, id_, "is in the set")
            None
        else:
            #print(id_, "is not in the set")
            keys_to_drop.append(region)

        #break    

    # Create a new dictionary without the specified keys
    total_cells_filtered = {key: value for key, value in total_cells_copy.items() if key not in keys_to_drop}

    # Concatenate all DataFrames into a single DataFrame
    concatenated_df = pd.concat(total_cells_filtered.values())

    # Group by all columns except the 'total_cells' column and sum the values
    summed_df = concatenated_df.groupby(concatenated_df.index).sum()
    print(f"The brain area {key} has {summed_df.shape[0]} different astrocyte types")
    #print("Total sum of these cells:", '{:,.0f}'.format(summed_df['density_mm3'].sum()))
    print("Total sum of these cells:", '{:,.0f}'.format(summed_df['total_cellnumber'].sum()))

The brain area wholebrain has 31 different astrocyte types
Total sum of these cells: 12,895,947
The brain area Cerebral cortex has 29 different astrocyte types
Total sum of these cells: 6,057,596
The brain area isocortex has 20 different astrocyte types
Total sum of these cells: 3,764,148
The brain area cerebellum has 15 different astrocyte types
Total sum of these cells: 667,961
The brain area fiber_tracts_ids has 29 different astrocyte types
Total sum of these cells: 1,699,063
The brain area hippocampus has 20 different astrocyte types
Total sum of these cells: 903,670
The brain area hippocampal_formation has 23 different astrocyte types
Total sum of these cells: 1,421,362
The brain area thalamus has 23 different astrocyte types
Total sum of these cells: 434,170
The brain area striatum has 21 different astrocyte types
Total sum of these cells: 910,815
The brain area VPL has 10 different astrocyte types
Total sum of these cells: 12,039
The brain area LGd has 9 different astrocyte type

In [23]:
#microglia
total_cells_copy = copy.deepcopy(total_cells) 

for key, df in total_cells_copy.items():
    total_cells_copy[key] = df[df.index.isin(microglia)]

for key, value in areas.items():
    
    keys_to_drop = ['unassigned', 'brain-unassigned']
    for region in regions:
        
        #filtered_data = region_info[(region_info['cluster_as_filename'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        filtered_data = region_info[(region_info['parcellation_term_acronym'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        id_ = filtered_data['label_numbers'].iloc[0]
        
        if int(id_) in value:
            #print(region, id_, "is in the set")
            None
        else:
            #print(id_, "is not in the set")
            keys_to_drop.append(region)

        #break    

    # Create a new dictionary without the specified keys
    total_cells_filtered = {key: value for key, value in total_cells_copy.items() if key not in keys_to_drop}

    # Concatenate all DataFrames into a single DataFrame
    concatenated_df = pd.concat(total_cells_filtered.values())

    # Group by all columns except the 'total_cells' column and sum the values
    summed_df = concatenated_df.groupby(concatenated_df.index).sum()
    print(f"The brain area {key} has {summed_df.shape[0]} different microglia types")
    #print("Total sum of these cells:", '{:,.0f}'.format(summed_df['density_mm3'].sum()))
    print("Total sum of these cells:", '{:,.0f}'.format(summed_df['total_cellnumber'].sum()))

The brain area wholebrain has 1 different microglia types
Total sum of these cells: 1,285,978
The brain area Cerebral cortex has 1 different microglia types
Total sum of these cells: 689,373
The brain area isocortex has 1 different microglia types
Total sum of these cells: 488,420
The brain area cerebellum has 1 different microglia types
Total sum of these cells: 46,695
The brain area fiber_tracts_ids has 1 different microglia types
Total sum of these cells: 138,837
The brain area hippocampus has 1 different microglia types
Total sum of these cells: 71,552
The brain area hippocampal_formation has 1 different microglia types
Total sum of these cells: 118,975
The brain area thalamus has 1 different microglia types
Total sum of these cells: 29,292
The brain area striatum has 1 different microglia types
Total sum of these cells: 123,053
The brain area VPL has 1 different microglia types
Total sum of these cells: 962
The brain area LGd has 1 different microglia types
Total sum of these cell

In [24]:
#oligos
total_cells_copy = copy.deepcopy(total_cells) 

for key, df in total_cells_copy.items():
    total_cells_copy[key] = df[df.index.isin(oligos)]

for key, value in areas.items():
    
    keys_to_drop = ['unassigned', 'brain-unassigned']
    for region in regions:
        
        #filtered_data = region_info[(region_info['cluster_as_filename'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        filtered_data = region_info[(region_info['parcellation_term_acronym'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        id_ = filtered_data['label_numbers'].iloc[0]
        
        if int(id_) in value:
            #print(region, id_, "is in the set")
            None
        else:
            #print(id_, "is not in the set")
            keys_to_drop.append(region)

        #break    

    # Create a new dictionary without the specified keys
    total_cells_filtered = {key: value for key, value in total_cells_copy.items() if key not in keys_to_drop}

    # Concatenate all DataFrames into a single DataFrame
    concatenated_df = pd.concat(total_cells_filtered.values())

    # Group by all columns except the 'total_cells' column and sum the values
    summed_df = concatenated_df.groupby(concatenated_df.index).sum()
    print(f"The brain area {key} has {summed_df.shape[0]} different oligocyte types")
    #print("Total sum of these cells:", '{:,.0f}'.format(summed_df['density_mm3'].sum()))
    print("Total sum of these cells:", '{:,.0f}'.format(summed_df['total_cellnumber'].sum()))

The brain area wholebrain has 23 different oligocyte types
Total sum of these cells: 13,595,084
The brain area Cerebral cortex has 23 different oligocyte types
Total sum of these cells: 2,777,282
The brain area isocortex has 22 different oligocyte types
Total sum of these cells: 1,663,332
The brain area cerebellum has 21 different oligocyte types
Total sum of these cells: 871,934
The brain area fiber_tracts_ids has 23 different oligocyte types
Total sum of these cells: 3,738,535
The brain area hippocampus has 22 different oligocyte types
Total sum of these cells: 331,604
The brain area hippocampal_formation has 23 different oligocyte types
Total sum of these cells: 667,532
The brain area thalamus has 23 different oligocyte types
Total sum of these cells: 577,229
The brain area striatum has 23 different oligocyte types
Total sum of these cells: 853,826
The brain area VPL has 18 different oligocyte types
Total sum of these cells: 38,140
The brain area LGd has 15 different oligocyte types

In [25]:
#exci_inhib_sum
total_cells_copy = copy.deepcopy(total_cells) 

for key, df in total_cells_copy.items():
    total_cells_copy[key] = df[df.index.isin(exci_inhib_sum)]

for key, value in areas.items():
    
    keys_to_drop = ['unassigned', 'brain-unassigned']
    for region in regions:
        
        #filtered_data = region_info[(region_info['cluster_as_filename'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        filtered_data = region_info[(region_info['parcellation_term_acronym'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        id_ = filtered_data['label_numbers'].iloc[0]
        
        if int(id_) in value:
            #print(region, id_, "is in the set")
            None
        else:
            #print(id_, "is not in the set")
            keys_to_drop.append(region)

        #break    

    # Create a new dictionary without the specified keys
    total_cells_filtered = {key: value for key, value in total_cells_copy.items() if key not in keys_to_drop}

    # Concatenate all DataFrames into a single DataFrame
    concatenated_df = pd.concat(total_cells_filtered.values())

    # Group by all columns except the 'total_cells' column and sum the values
    summed_df = concatenated_df.groupby(concatenated_df.index).sum()
    print(f"The brain area {key} has {summed_df.shape[0]} different INH+EXC(Sum) types")
    #print("Total sum of these cells:", '{:,.0f}'.format(summed_df['density_mm3'].sum()))
    print("Total sum of these cells:", '{:,.0f}'.format(summed_df['total_cellnumber'].sum()))

The brain area wholebrain has 5080 different INH+EXC(Sum) types
Total sum of these cells: 51,869,637
The brain area Cerebral cortex has 2944 different INH+EXC(Sum) types
Total sum of these cells: 29,119,517
The brain area isocortex has 870 different INH+EXC(Sum) types
Total sum of these cells: 19,101,627
The brain area cerebellum has 770 different INH+EXC(Sum) types
Total sum of these cells: 3,543,172
The brain area fiber_tracts_ids has 2998 different INH+EXC(Sum) types
Total sum of these cells: 2,472,154
The brain area hippocampus has 508 different INH+EXC(Sum) types
Total sum of these cells: 2,324,177
The brain area hippocampal_formation has 1633 different INH+EXC(Sum) types
Total sum of these cells: 3,887,649
The brain area thalamus has 1588 different INH+EXC(Sum) types
Total sum of these cells: 1,829,618
The brain area striatum has 1297 different INH+EXC(Sum) types
Total sum of these cells: 7,114,754
The brain area VPL has 41 different INH+EXC(Sum) types
Total sum of these cells: 5